# CMI Gesture Recognition - Inference Notebook

This notebook demonstrates inference with the trained model, handling sequence chunking and aggregation for predictions.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import yaml
from collections import defaultdict

from src.dataset import CMIDataset, SequenceProcessor, prepare_gesture_labels
from src.model import create_model
from src.trainer import CMITrainer

print(f"Project root: {project_root}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Load Configuration and Model

In [ ]:
# Load config
with open(project_root / 'config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"- Model d_model: {config['model']['d_model']}")
print(f"- Model layers: {config['model']['num_layers']}")
print(f"- Max sequence length for chunking: {config['data']['max_seq_length']}")
print(f"- Max sequence length for positional encoding: {config['model']['max_seq_length']}")

In [ ]:
# Find the best trained model
experiment_dirs = list((project_root / 'experiments').glob('cmi_training_*'))
if not experiment_dirs:
    raise FileNotFoundError("No training experiments found")

# Use the most recent experiment
latest_experiment = max(experiment_dirs, key=lambda x: x.name)
model_path = latest_experiment / 'models' / 'best_model.pt'

print(f"Loading model from: {model_path}")

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(model_path, map_location=device)

# Create model with saved configuration
model_config = checkpoint['model_config']
model = create_model(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Model loaded successfully on {device}")
print(f"Model config: {model_config}")

# Load label encoder if available
label_encoder = checkpoint.get('label_encoder')
if label_encoder is None:
    print("Warning: No label encoder found in checkpoint")
else:
    print(f"Label encoder loaded with {len(label_encoder.classes_)} classes")

## Load Test Data

In [ ]:
# Load test data
test_df = pl.read_csv(project_root / 'dataset' / 'test.csv')
test_demographics = pl.read_csv(project_root / 'dataset' / 'test_demographics.csv')

# Merge with demographics if needed
if 'participant_id' in test_df.columns and 'participant_id' in test_demographics.columns:
    test_df = test_df.join(test_demographics, on='participant_id', how='left')

print(f"Test data shape: {test_df.shape}")
print(f"Unique sequences: {test_df['sequence_id'].n_unique()}")
print(f"Columns: {test_df.columns}")

## Define Inference Functions

In [ ]:
class InferenceProcessor:
    """Process sequences for inference with proper chunking and aggregation."""
    
    def __init__(self, model, device, max_chunk_length=20):
        self.model = model
        self.device = device
        self.max_chunk_length = max_chunk_length
        self.sequence_processor = SequenceProcessor()
    
    def process_sequence_for_inference(self, sequence_data):
        """Process a single sequence, creating chunks if necessary."""
        sequence_id = sequence_data['sequence_id'][0]
        
        try:
            # Create enhanced features using FeatureProcessor
            enhanced_features = self.sequence_processor.feature_processor.create_sequence_features(
                sequence_data
            )
            
            # Apply chunking for inference
            chunks = self._chunk_sequence_for_inference(
                enhanced_features, sequence_id, self.max_chunk_length
            )
            
        except Exception as e:
            print(f"Error processing enhanced features for {sequence_id}, falling back to original: {e}")
            # Fallback to original processing
            acc_cols = ["acc_x", "acc_y", "acc_z"]
            rot_cols = ["rot_w", "rot_x", "rot_y", "rot_z"]
            thm_cols = [f"thm_{i}" for i in range(1, 6)]
            tof_cols = [f"tof_{i}_v{j}" for i in range(1, 6) for j in range(64)]
            
            seq_data = sequence_data.select(
                acc_cols + rot_cols + thm_cols + tof_cols
            ).to_numpy()
            
            chunks = self._chunk_original_sequence_for_inference(
                seq_data, sequence_id, self.max_chunk_length
            )
        
        return chunks
    
    def _chunk_sequence_for_inference(self, enhanced_features, sequence_id, max_seq_length):
        """Chunk enhanced features for inference."""
        tof_data = enhanced_features["tof"]
        acc_data = enhanced_features["acc"]
        rot_data = enhanced_features["rot"]
        thm_data = enhanced_features["thm"]
        
        seq_length = len(tof_data)
        chunks = []
        
        if seq_length <= max_seq_length:
            # No chunking needed
            chunks.append({
                "sequence_id": sequence_id,
                "original_sequence_id": sequence_id,
                "enhanced_data": enhanced_features,
                "chunk_start_idx": 0,
            })
        else:
            # Split into chunks
            num_chunks = (seq_length + max_seq_length - 1) // max_seq_length
            for i in range(num_chunks):
                start_idx = i * max_seq_length
                end_idx = min((i + 1) * max_seq_length, seq_length)
                
                chunk_features = {
                    "tof": tof_data[start_idx:end_idx],
                    "acc": acc_data[start_idx:end_idx],
                    "rot": rot_data[start_idx:end_idx],
                    "thm": thm_data[start_idx:end_idx],
                }
                
                chunks.append({
                    "sequence_id": f"{sequence_id}_chunk_{i}",
                    "original_sequence_id": sequence_id,
                    "enhanced_data": chunk_features,
                    "chunk_start_idx": start_idx,
                })
        
        return chunks
    
    def _chunk_original_sequence_for_inference(self, seq_data, sequence_id, max_seq_length):
        """Chunk original sequence data for inference."""
        seq_length = len(seq_data)
        chunks = []
        
        if seq_length <= max_seq_length:
            # No chunking needed
            chunks.append({
                "sequence_id": sequence_id,
                "original_sequence_id": sequence_id,
                "data": seq_data,
                "chunk_start_idx": 0,
            })
        else:
            # Split into chunks
            num_chunks = (seq_length + max_seq_length - 1) // max_seq_length
            for i in range(num_chunks):
                start_idx = i * max_seq_length
                end_idx = min((i + 1) * max_seq_length, seq_length)
                
                chunks.append({
                    "sequence_id": f"{sequence_id}_chunk_{i}",
                    "original_sequence_id": sequence_id,
                    "data": seq_data[start_idx:end_idx],
                    "chunk_start_idx": start_idx,
                })
        
        return chunks
    
    def predict_sequences(self, test_df, batch_size=16):
        """Predict on test sequences with proper aggregation."""
        # Group by sequence_id
        grouped = test_df.group_by("sequence_id")
        
        # Process all sequences into chunks
        all_chunks = []
        sequence_to_chunks = defaultdict(list)
        
        print("Processing sequences into chunks...")
        for seq_id, group in tqdm(grouped):
            chunks = self.process_sequence_for_inference(group)
            for chunk in chunks:
                all_chunks.append(chunk)
                sequence_to_chunks[chunk["original_sequence_id"]].append(len(all_chunks) - 1)
        
        print(f"Created {len(all_chunks)} chunks from {len(sequence_to_chunks)} sequences")
        
        # Create dataset and dataloader
        dataset = CMIDataset(
            all_chunks,
            max_length=self.max_chunk_length
        )
        
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0
        )
        
        # Run inference on all chunks
        chunk_predictions = []
        chunk_probabilities = []
        
        print("Running inference on chunks...")
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(dataloader):
                # Move to device
                tof_data = batch["tof"].to(self.device)
                acc_data = batch["acc"].to(self.device)
                rot_data = batch["rot"].to(self.device)
                thm_data = batch["thm"].to(self.device)
                chunk_start_idx = batch.get("chunk_start_idx")
                if chunk_start_idx is not None:
                    chunk_start_idx = chunk_start_idx.to(self.device)
                
                # Forward pass
                outputs = self.model(
                    tof_data, acc_data, rot_data, thm_data, chunk_start_idx
                )
                
                # Get probabilities and predictions
                probabilities = F.softmax(outputs, dim=1)
                predictions = torch.argmax(outputs, dim=1)
                
                chunk_predictions.extend(predictions.cpu().numpy())
                chunk_probabilities.extend(probabilities.cpu().numpy())
        
        # Aggregate predictions by original sequence_id
        print("Aggregating predictions by sequence...")
        sequence_predictions = {}
        
        for original_seq_id, chunk_indices in sequence_to_chunks.items():
            # Get probabilities for all chunks of this sequence
            chunk_probs = [chunk_probabilities[i] for i in chunk_indices]
            
            # Average probabilities across chunks
            avg_probs = np.mean(chunk_probs, axis=0)
            
            # Get final prediction
            final_prediction = np.argmax(avg_probs)
            
            sequence_predictions[original_seq_id] = {
                'prediction': final_prediction,
                'probabilities': avg_probs,
                'num_chunks': len(chunk_indices)
            }
        
        return sequence_predictions

# Initialize inference processor
inference_processor = InferenceProcessor(
    model=model, 
    device=device, 
    max_chunk_length=config['data']['max_seq_length']
)

print("Inference processor initialized")

## Run Inference on Test Data

In [ ]:
# Run inference
predictions = inference_processor.predict_sequences(test_df, batch_size=16)

print(f"Generated predictions for {len(predictions)} sequences")

# Show some example predictions
for i, (seq_id, pred_info) in enumerate(list(predictions.items())[:5]):
    print(f"Sequence {seq_id}:")
    print(f"  Prediction: {pred_info['prediction']}")
    print(f"  Max probability: {pred_info['probabilities'].max():.4f}")
    print(f"  Number of chunks: {pred_info['num_chunks']}")
    if label_encoder is not None:
        gesture_name = label_encoder.inverse_transform([pred_info['prediction']])[0]
        print(f"  Gesture: {gesture_name}")
    print()

## Analysis and Results

In [ ]:
# Analyze prediction distribution
prediction_counts = defaultdict(int)
chunk_counts = defaultdict(int)
confidence_scores = []

for seq_id, pred_info in predictions.items():
    prediction_counts[pred_info['prediction']] += 1
    chunk_counts[pred_info['num_chunks']] += 1
    confidence_scores.append(pred_info['probabilities'].max())

print("Prediction distribution:")
for pred, count in sorted(prediction_counts.items()):
    if label_encoder is not None:
        gesture_name = label_encoder.inverse_transform([pred])[0]
        print(f"  Class {pred} ({gesture_name}): {count} sequences")
    else:
        print(f"  Class {pred}: {count} sequences")

print("\nChunk distribution:")
for num_chunks, count in sorted(chunk_counts.items()):
    print(f"  {num_chunks} chunks: {count} sequences")

print(f"\nConfidence statistics:")
print(f"  Mean confidence: {np.mean(confidence_scores):.4f}")
print(f"  Median confidence: {np.median(confidence_scores):.4f}")
print(f"  Min confidence: {np.min(confidence_scores):.4f}")
print(f"  Max confidence: {np.max(confidence_scores):.4f}")

## Save Predictions

In [ ]:
# Create submission dataframe
submission_data = []

for seq_id, pred_info in predictions.items():
    submission_data.append({
        'sequence_id': seq_id,
        'prediction': pred_info['prediction'],
        'confidence': pred_info['probabilities'].max(),
        'num_chunks': pred_info['num_chunks']
    })
    
    # Add probability columns if needed
    for class_idx, prob in enumerate(pred_info['probabilities']):
        submission_data[-1][f'prob_class_{class_idx}'] = prob

# Convert to DataFrame
submission_df = pd.DataFrame(submission_data)

# Save predictions
output_dir = project_root / 'predictions'
output_dir.mkdir(exist_ok=True)

submission_path = output_dir / 'test_predictions.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Predictions saved to: {submission_path}")
print(f"Submission shape: {submission_df.shape}")

# Display first few predictions
display_cols = ['sequence_id', 'prediction', 'confidence', 'num_chunks']
if label_encoder is not None:
    submission_df['gesture_name'] = label_encoder.inverse_transform(submission_df['prediction'])
    display_cols.append('gesture_name')

print("\nFirst 10 predictions:")
print(submission_df[display_cols].head(10))

## Summary

This notebook demonstrates:

1. **Proper sequence chunking**: Test sequences are split into chunks of the same length used during training
2. **Chunk-aware inference**: Each chunk is processed with appropriate positional encoding based on its position in the original sequence
3. **Prediction aggregation**: Multiple chunk predictions for the same sequence are aggregated by averaging probabilities and taking the argmax
4. **Confidence estimation**: The maximum probability across classes provides a confidence measure

Key differences from training:
- During training: sequences are chunked and each chunk gets a separate label
- During inference: sequences are chunked but predictions are aggregated back to the original sequence level

This approach ensures that long sequences are handled consistently between training and inference while properly accounting for sequence structure.